<a href="https://colab.research.google.com/github/1matheeus/ClubedeIA/blob/main/ClubedeIA_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd

# Especifique o caminho do seu arquivo Excel de entrada
# Substitua 'seu_arquivo.xlsx' pelo nome do seu arquivo Excel
input_excel_file = 'formsTeste.xlsx'

# Especifique o caminho para o arquivo CSV de saída
# Substitua 'seu_arquivo_convertido.csv' pelo nome que você deseja para o arquivo CSV
output_csv_file = 'RespostasFormulario.csv'

try:
    # Ler o arquivo Excel em um DataFrame do pandas
    df = pd.read_excel(input_excel_file)

    df = df.drop(df.columns[0], axis=1)  # Remove a primeira coluna
    df = df.drop(df.columns[1], axis=1)  # Remove a segunda coluna

    # Obtém o nome da primeira coluna
    first_column = df.columns[0]

    # Cria uma nova lista de colunas com a primeira coluna no final
    new_column_order = list(df.columns[1:]) + [first_column]

    # Reindexa o DataFrame com a nova ordem
    df = df.reindex(columns=new_column_order)

    df.columns = [f"Q{i+1}" for i in range(33,)] # Corrected to assign 33 column names
    df.rename(columns={'Q33': 'R'}, inplace=True)

    mapa_r = {
        "31TE": 31,
        "32BI": 32,
        "33EL": 33,
        "34DS": 34
    }

    mapa_q2 = { #faz um Remap de cada resposta para um valor numérico
    "Matemática – por causa da lógica e resolução de problemas": 1,
    "Biologia – por curiosidade sobre a vida e o corpo humano": 2,
    "Física – pela compreensão de fenômenos e aplicações práticas": 3,
    "Informática – pelo uso da tecnologia no dia a dia": 4,
    "Nenhuma em especial – preferia atividades mais práticas ou aplicadas": 5
    }

    mapa_q7 = {
    "Um hospital ou clínica, auxiliando na área da saúde com tecnologia": 1,
    "Uma startup ou empresa de tecnologia, criando inovações digitais": 2,
    "Uma indústria automatizada, lidando com máquinas e processos técnicos": 3,
    "Um provedor de internet ou empresa de telecomunicações, garantindo a conectividade": 4
    }

    mapa_q25 = {
    "Raciocínio lógico e solução de problemas objetivos": 1,
    "Comunicação clara e trabalho em equipe": 2,
    "Criatividade para inovar e pensar fora da caixa": 3,
    "Precisão, paciência e atenção aos mínimos detalhes": 4
    }

    df["Q2"] = df["Q2"].map(mapa_q2) #substitui os valores antigos de respostas em string para o novo mapa com respostas em int
    df["Q7"] = df["Q7"].map(mapa_q7)
    df["Q25"] = df["Q25"].map(mapa_q25)
    df["R"] = df["R"].map(mapa_r).astype("Int64")

    df.to_csv(output_csv_file, index=False)

    print(f"Arquivo '{input_excel_file}' convertido com sucesso para '{output_csv_file}'")
    print(f"A segunda coluna foi renomeada para '{first_column}' e movida para o final.")

except FileNotFoundError:
    print(f"Erro: O arquivo '{input_excel_file}' não foi encontrado. Por favor, verifique o caminho do arquivo.")
except IndexError:
    print("Erro: Não há uma segunda coluna no arquivo Excel para renomear e mover.")
except Exception as e:
    print(f"Ocorreu um erro durante a conversão: {e}")

Arquivo 'formsTeste.xlsx' convertido com sucesso para 'RespostasFormulario.csv'
A segunda coluna foi renomeada para 'Qual é a sua turma ? ' e movida para o final.


In [45]:
from sklearn.model_selection import train_test_split

# Definir as colunas de features (X) e a coluna target (y)
# Com base no DataFrame 'df' carregado anteriormente, as colunas de features são 'Q1' a 'Q32'.
feature_columns = [f'Q{i}' for i in range(1, 33)]
target_column = 'R'

# Verificar se as colunas de features e target existem no DataFrame
missing_columns = [col for col in feature_columns if col not in df.columns]
if target_column not in df.columns:
    missing_columns.append(target_column)

if missing_columns:
    print(f"Erro: As seguintes colunas não foram encontradas no DataFrame: {missing_columns}")
else:
    X = df[feature_columns]
    y = df[target_column]

    # Dividir os dados em conjuntos de treino e teste (80% treino, 20% teste)
    # random_state garante que a divisão seja a mesma toda vez que você rodar o código
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    print("Dados divididos em conjuntos de treino e teste com sucesso.")
    print("Tamanho do conjunto de treino (features):", X_train.shape)
    print("Tamanho do conjunto de teste (features):", X_test.shape)
    print("Tamanho do conjunto de treino (target):", y_train.shape)
    print("Tamanho do conjunto de teste (target):", y_test.shape)

    # Opcional: Mostrar as primeiras linhas dos conjuntos resultantes
    # print("\nPrimeiras 5 linhas de X_train:")
    # display(X_train.head())
    # print("\nPrimeiras 5 linhas de y_train:")
    # display(y_train.head())

Dados divididos em conjuntos de treino e teste com sucesso.
Tamanho do conjunto de treino (features): (41, 32)
Tamanho do conjunto de teste (features): (11, 32)
Tamanho do conjunto de treino (target): (41,)
Tamanho do conjunto de teste (target): (11,)


In [46]:
from sklearn.tree import DecisionTreeClassifier

# Instanciar o classificador Decision Tree
# Você pode ajustar o max_depth para controlar a complexidade da árvore
model = DecisionTreeClassifier(random_state=42) # Adicionado random_state para reprodutibilidade

# Treinar o modelo com os dados de treino
# Certifique-se de que X_train e y_train foram definidos nos passos anteriores
try:
    model.fit(X_train, y_train)
    print("Modelo Decision Tree treinado com sucesso.")
except NameError:
    print("Erro: X_train ou y_train não foram definidos. Certifique-se de que os passos anteriores para carregar e dividir os dados foram executados com sucesso.")
except Exception as e:
    print(f"Ocorreu um erro durante o treinamento do modelo: {e}")

Modelo Decision Tree treinado com sucesso.


In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fazer predições no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o desempenho do modelo
acuracia = accuracy_score(y_test, y_pred)
precisao = precision_score(y_test, y_pred, average='weighted', zero_division=1) # Use 'weighted' para lidar com classes desbalanceadas
recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)

print(f"Acurácia do modelo: {acuracia:.2f}")
print(f"Precisão do modelo: {precisao:.2f}")
print(f"Recall do modelo: {recall:.2f}")
print(f"F1-score do modelo: {f1:.2f}")

Acurácia do modelo: 0.64
Precisão do modelo: 0.67
Recall do modelo: 0.64
F1-score do modelo: 0.63


In [48]:
# Salvar o DataFrame em CSV no Colab
df.to_csv("meu_dataframe.csv", index=False)

print("Arquivo salvo como 'meu_dataframe.csv' na pasta atual do Colab")


Arquivo salvo como 'meu_dataframe.csv' na pasta atual do Colab


In [49]:
# Carregar o CSV
file_path = "/content/meu_dataframe.csv"
df = pd.read_csv(file_path)

# Selecionar as colunas das questões
perguntas = [col for col in df.columns if col.startswith("Q")]

# Agrupar por R e calcular média e desvio padrão
estatisticas = df.groupby("R")[perguntas].agg(["mean", "std"])

# Renomear colunas para clareza
estatisticas.columns = [f"{col}_{stat}" for col, stat in estatisticas.columns]

# Mostrar resultado
print(estatisticas)


     Q1_mean    Q1_std   Q2_mean    Q2_std   Q3_mean    Q3_std   Q4_mean  \
R                                                                          
31  3.111111  1.054093  3.333333  1.500000  4.666667  0.500000  4.222222   
32  3.416667  0.996205  2.166667  0.937437  4.750000  0.621582  3.083333   
33  3.285714  1.380131  3.000000  1.732051  4.571429  1.133893  4.571429   
34  3.041667  1.122078  3.458333  1.178767  4.250000  1.293798  3.583333   

      Q4_std   Q5_mean    Q5_std  ...  Q28_mean   Q28_std  Q29_mean   Q29_std  \
R                                 ...                                           
31  0.971825  3.777778  1.394433  ...  4.555556  0.726483  4.666667  0.707107   
32  1.564279  3.250000  1.055290  ...  3.666667  1.556998  4.500000  0.674200   
33  0.534522  4.142857  1.069045  ...  4.571429  0.534522  5.000000  0.000000   
34  1.248187  3.666667  1.340560  ...  4.083333  1.017955  4.375000  1.209114   

    Q30_mean   Q30_std  Q31_mean   Q31_std  Q32_mean   Q

In [50]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# === 1. Carregar o CSV ===
file_path = "/content/meu_dataframe.csv"
df = pd.read_csv(file_path)

# === 2. Tratar dados ===
df = df.dropna()  # remove linhas com valores faltantes

# === 3. Separar features (Q1..Q32) e target (R) ===
X = df[[col for col in df.columns if col.lower().startswith("q")]]
y = df["R"]

# Converter target se for texto
if y.dtype == "object":
    le = LabelEncoder()
    y = le.fit_transform(y)

# === 4. Treinar modelo Random Forest ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Avaliar
y_pred = model.predict(X_test)
print("Relatório de classificação:\n")
print(classification_report(y_test, y_pred))

# === 5. Importância das features ===
importances = pd.DataFrame({
    "Pergunta": X.columns,
    "Importância": model.feature_importances_
}).sort_values(by="Importância", ascending=False)

# Mostrar top 10
print("\nTop 10 perguntas mais relevantes:")
print(importances.head(10))

# === 6. Salvar ranking completo em CSV ===
importances.to_csv("ranking_importancia_perguntas.csv", index=False)
print("\nArquivo 'ranking_importancia_perguntas.csv' salvo com sucesso.")

Relatório de classificação:

              precision    recall  f1-score   support

          31       0.00      0.00      0.00         3
          32       1.00      0.17      0.29         6
          33       0.00      0.00      0.00         1
          34       0.40      1.00      0.57         6

    accuracy                           0.44        16
   macro avg       0.35      0.29      0.21        16
weighted avg       0.53      0.44      0.32        16


Top 10 perguntas mais relevantes:
   Pergunta  Importância
6        Q7     0.104966
1        Q2     0.058411
9       Q10     0.047855
3        Q4     0.046860
22      Q23     0.039944
29      Q30     0.039417
7        Q8     0.036016
25      Q26     0.035686
14      Q15     0.035545
13      Q14     0.034464

Arquivo 'ranking_importancia_perguntas.csv' salvo com sucesso.


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [51]:
import pandas as pd
import numpy as np

# === 1. Carregar o CSV original ===
file_path = "/content/meu_dataframe.csv"  # ajuste se necessário
df = pd.read_csv(file_path)

# === 2. Identificar perguntas ===
perguntas = [col for col in df.columns if col.startswith("Q")]

# === 3. Calcular médias e desvios padrão por R ===
stats = df.groupby("R")[perguntas].agg(["mean", "std"])

# === 4. Gerar 200 respostas artificiais ===
n_sinteticas = 1000
sinteticos = []

for _ in range(n_sinteticas):
    # Escolhe aleatoriamente uma classe R existente
    r_value = np.random.choice(df["R"].unique())

    # Para cada questão, gerar resposta ~ Normal(mean, std) dessa classe
    respostas = {}
    for q in perguntas:
        mean = stats.loc[r_value, (q, "mean")]
        std = stats.loc[r_value, (q, "std")]
        if np.isnan(std) or std == 0:
            std = 0.5  # evitar problemas com std=0

        # Sorteia valor
        val = np.random.normal(mean, std)
        # Arredonda para inteiro e mantém entre 1 e 5
        val = int(np.clip(round(val), 1, 5))
        respostas[q] = val

    respostas["R"] = r_value
    sinteticos.append(respostas)

df_sintetico = pd.DataFrame(sinteticos)

# === 5. Concatenar com original ===
df_final = pd.concat([df, df_sintetico], ignore_index=True)

# === 6. Salvar em CSV ===
output_file = "dados_com_aumento.csv"
df_final.to_csv(output_file, index=False)

print(f"Novo dataset salvo como {output_file} com {len(df_final)} linhas (incluindo {n_sinteticas} artificiais).")


Novo dataset salvo como dados_com_aumento.csv com 1052 linhas (incluindo 1000 artificiais).
